# Network Analysis for ABCD data

http://dx.plos.org/10.1371/journal.pbio.1002328  
https://www.sciencedirect.com/science/article/pii/S105381191730109X?via%3Dihub

In [ ]:
# ! pip install python-louvain

In [1]:
import glob
import os
import networkx as nx
import numpy as np
import pandas as pd
import community
from sklearn.metrics.cluster import normalized_mutual_info_score
import bz2
import pickle
import pdb
import statistics
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt

from visbrain.objects import ConnectObj, SceneObj, SourceObj, BrainObj
from visbrain.io import download_file

import bct
import pdb

/Users/gracer/.local/lib/python3.6/site-packages/vispy/visuals/isocurve.py:22: UserWarning: VisPy is not yet compatible with matplotlib 2.2+
  warnings.warn("VisPy is not yet compatible with matplotlib 2.2+")


## Data of interest (from R notebook)

### Read in labels

In [ ]:
labels = pd.read_csv('/Users/gracer/Google Drive/ABCD/important_txt/locations.csv', sep=",")

In [ ]:
interest =  pd.read_csv('/Users/gracer/Google Drive/ABCD/important_txt/data4analysis.txt', sep=" ", header=None, 
                 index_col=False)
interest.head()

In [ ]:
subs=interest[0]
subs[1:10]

## Get data

In [ ]:
data = glob.glob('/Users/gracer/Google Drive/ABCD/ABCDworking/sub-*/keep/sub-NDAR*_ses-baselineYear1Arm1_task-rest_run-0*_bold_brain_norm_r_matrix.csv')






In [ ]:
data[0:10]

#### Make a dictonary to store the path and subject ID

In [ ]:
my_dict={}
for item in data:
    name=item.split('/')[6]
    print(name)
    my_dict.setdefault(name, []).append(item)

### Make a dictonary of all the file paths common to the subject list 

In [ ]:
path_dict = {k: my_dict[k] for k in subs if k in my_dict}
print(my_dict)

### Make a dictonary to read in the files

In [ ]:
data_dict={}
num=[]

for key, value in path_dict.items():
    num.append(len(value))
    for i in value:
        x=pd.read_csv(i, header=None,index_col=False)
#         data_dict.setdefault(key, x)
        data_dict.setdefault(key, []).append(pd.read_csv(i, header=None,index_col=False))
#         data_dict[key]= pd.read_csv(i, header=None,index_col=False)

In [ ]:
np.mean(num)

In [ ]:
len(list(data_dict.keys()))

## Function to create dictionary of covariates

In [ ]:
interest_dict=interest.set_index(interest[0]).to_dict()
def removekey(d, key):
    r = dict(d)
    del r[key]
    return r
cov_dict=removekey(interest_dict, 0)
cov_dict['sex'] = cov_dict.pop(1)
cov_dict['PCS'] = cov_dict.pop(2)
cov_dict['OVOB'] = cov_dict.pop(3)
# cov_dict['BMItile'] = cov_dict.pop(4)
# cov_dict['PDSscore'] = cov_dict.pop(5)
# cov_dict['age'] = cov_dict.pop(6)
print(cov_dict.keys())

## Function to create graph objects

In [ ]:
def create_corr_network_5(G, corr_direction, min_correlation):
    ##Creates a copy of the graph
    H = G.copy()
    
    ##Checks all the edges and removes some based on corr_direction
    for stock1, stock2, weight in list(G.edges(data=True)):
        ##if we only want to see the positive correlations we then delete the edges with weight smaller than 0        
        if corr_direction == "positive":
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] <0 or weight["weight"] < min_correlation:
                H.remove_edge(stock1, stock2)
        ##this part runs if the corr_direction is negative and removes edges with weights equal or largen than 0
        else:
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] >=0 or weight["weight"] > min_correlation:
                H.remove_edge(stock1, stock2)
    return(H)


### Function to make a graph object BY SUBJECT
This will return: 
* The edges (noramlized R correlation matrix, in pandas dataframe)
* The mean_FC (the mean functional connectivity per subject/node)
* The graphs (this will contain the raw graph object G as well as the the partion values from the modularity calculation)
* The mu is the mean of all the runs into a single correlation matrix per subject

In [ ]:
def make_graphs(list_o_data, direction, min_cor):
    edge_dict={}
    FC_dict={}
    graph_dict={}
    
    print(len(list(list_o_data.keys())))
    j=0
    mylist=[]
    mu_network={}
    bad={}
    for key, val_list in list_o_data.items():
        print("on number %s"%(str(j)))
        j=j+1
        newlist=[]
        for item in val_list:
            
            print(np.array(item).diagonal())
            if np.all(np.array(item).diagonal()) == True:
                newlist.append(np.array(item))
                i=item.set_index(labels.ID)
                i.rename(columns=labels.ID, inplace=True)
                edge_dict.setdefault(key, []).append(i)
                
            else:
                print("%s is fucked"%key)
                bad.setdefault(key, []).append(item)
                
                
        try:        
            y=np.dstack(newlist)
            print(y.shape)
            y=np.rollaxis(y,-1)
            print(y.shape)
            mu=np.mean(y, axis=0)
            print(np.array(mu).diagonal())
            mu_network.setdefault(key, mu)

            m=x.mean()
            FC_dict.setdefault(key, []).append(m)

            G = nx.from_numpy_matrix(mu)
            for i, nlrow in labels.iterrows():
                G.node[i].update(nlrow[0:].to_dict())

            graph_dict.setdefault(key, []).append(G)

            partition = community.best_partition(create_corr_network_5(G, direction,min_cor))

            graph_dict.setdefault(key, []).append(partition)
        except ValueError:
            continue
        
        
#         pdb.set_trace()
              
    return({'edges':edge_dict, 'correlations':cor_dict, 'mean_FC':FC_dict, 'graphs':graph_dict, 'mu':mu_network})

In [ ]:
GRAPHS=make_graphs(data_dict, "positive", 0)

In [ ]:
len(list(GRAPHS['mu']))

In [ ]:
for key in cov_dict.keys():
    for subkey, value in cov_dict[key].items():
        if subkey in GRAPHS['graphs']:
            GRAPHS['graphs'][subkey].append(value)
            print(GRAPHS['graphs'][subkey])

In [ ]:
len(list(GRAPHS['graphs']))

In [ ]:
cov_dict['PCS']['sub-NDARINV019DXLU4']

In [ ]:
GRAPHS.keys()

In [ ]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

In [ ]:
z = merge_two_dicts(GRAPHS, interest_dict)

In [ ]:
z.keys()

In [ ]:
z['sex'] =z.pop(0)
z['sex'] =z.pop(1)
z['PCS'] =z.pop(2)
z['OVOB'] =z.pop(3)
# z['BMItile'] =z.pop(4)
# z['PDSscore'] =z.pop(5)
# z['age'] =z.pop(6)

### How to delete items in list that you accidentally made

In [ ]:
# for key,value in GRAPHS['mean_FC'].items():
#      del value[-7:]

# Differences in modularity by subject

### Participation coefficient 
#### Parameters
    ----------
    W : NxN np.ndarray
        binary/weighted directed/undirected connection matrix
    ci : Nx1 np.ndarray
        community affiliation vector
    degree : str
        Flag to describe nature of graph 'undirected': For undirected graphs
                                         'in': Uses the in-degree
                                         'out': Uses the out-degree

In [ ]:
def participation_award(cor_mats, parts):
#     cor_mats need to be something like a dictionary of correlation matrices with the subject as the key
#     parts need to be the numerical modularity values
    allPC={}
    for keys, values in cor_mats.items():
        print(keys)
        cor_mat = np.array(values)
        test_array=np.array(list(list(z['graphs'][keys])[1].values()))
        testPART=np.vstack(test_array)

        PC=bct.participation_coef(W=cor_mat, ci= testPART)
        allPC[keys]=PC
        
    return(allPC)


In [ ]:
allPC=participation_award(z['mu'],z['graphs'])

### Clustering Coefficient 
The weighted clustering coefficient is the average "intensity" of
    triangles around a node.
   #### Parameters
    ----------
    W : NxN np.ndarray
        weighted directed connection matrix
    Returns
    -------
    C : Nx1 np.ndarray
        clustering coefficient vector
    Notes
    -----
    Methodological note (also see clustering_coef_bd)
    The weighted modification is as follows:
    - The numerator: adjacency matrix is replaced with weights matrix ^ 1/3
    - The denominator: no changes from the binary version
    The above reduces to symmetric and/or binary versions of the clustering
    coefficient for respective graphs.

In [ ]:
z.keys()

In [ ]:
def cluster_fuq(cor_mats):
    clusters={}
    for keys, values in cor_mats.items():
        CC=bct.clustering_coef_wd(values)
        clusters[keys]=CC
    return(clusters)
#         pdb.set_trace()

In [ ]:
allCC=cluster_fuq(z['mu'])

### UR SO Random
def null_model_und_sign(W, bin_swaps=5, wei_freq=.1, seed=None):
    '''
    This function randomizes an undirected network with positive and
    negative weights, while preserving the degree and strength
    distributions. This function calls randmio_und.m
#### Parameters
    ----------
    W : NxN np.ndarray
        undirected weighted connection matrix
    bin_swaps : int
        average number of swaps in each edge binary randomization. Default
        value is 5. 0 swaps implies no binary randomization.
    wei_freq : float
        frequency of weight sorting in weighted randomization. 0<=wei_freq<1.
        wei_freq == 1 implies that weights are sorted at each step.
        wei_freq == 0.1 implies that weights sorted each 10th step (faster,
            default value)
        wei_freq == 0 implies no sorting of weights (not recommended)
    seed : hashable, optional
        If None (default), use the np.random's global random state to generate random numbers.
        Otherwise, use a new np.random.RandomState instance seeded with the given value.
    Returns
    -------
    W0 : NxN np.ndarray
        randomized weighted connection matrix
    R : 4-tuple of floats
        Correlation coefficients between strength sequences of input and
        output connection matrices, rpos_in, rpos_out, rneg_in, rneg_out
    Notes
    -----
    The value of bin_swaps is ignored when binary topology is fully
        connected (e.g. when the network has no negative weights).
    Randomization may be better (and execution time will be slower) for
        higher values of bin_swaps and wei_freq. Higher values of bin_swaps
        may enable a more random binary organization, and higher values of
        wei_freq may enable a more accurate conservation of strength
        sequences.
    R are the correlation coefficients between positive and negative
        strength sequences of input and output connection matrices and are
        used to evaluate the accuracy with which strengths were preserved.
        Note that correlation coefficients may be a rough measure of
        strength-sequence accuracy and one could implement more formal tests
        (such as the Kolmogorov-Smirnov test) if desired.
    '''

In [ ]:
def UR_SO_RANDOM(cor_mat):
    nullz={}
    for keys, values in cor_mat.items():
        print(keys)
        test=bct.null_model_und_sign(values, bin_swaps=5, wei_freq=.1)
        pdb.set_trace()

In [ ]:
UR_SO_RANDOM(z['mu'])

In [ ]:
null_model_und_sign(W, bin_swaps=5, wei_freq=.1, seed=None)

### Normalized information score  
sklearn.metrics.normalized_mutual_info_score(labels_true [group 1], labels_pred [group 2], average_method=’warn’)

In [ ]:
diff_no_ov=normalized_mutual_info_score(norm_max, ov_max)

In [ ]:
diff_no_ob=normalized_mutual_info_score(norm_max, ob_max)

In [ ]:
diff_ov_ob=normalized_mutual_info_score(ov_max, ob_max)

## Parcelation 
Through BIAC https://wiki.biac.duke.edu/biac:analysis:resting_pipeline

## Individual and Group Matrices
Network-level analysis will be performed with inividual correltion matrices

## Thresholding
In accordance with van den Heuvel et al. 2017, we will examine and test statistical differences in functional connectivity (FC) defined as the mean of the correlation matrix. FC will be included in statistical tests between groups.

## Partitioning
Will partition full 264 connectome into modules using louvain algorithm. 

## Check the partition
Will use normalized mutual information to assess similarity between network assignments. NMI measures information shared between two probability distribution functions, specifically measuring how much knowing one distribution leads to certainty ofthe other. Permuted the labels of individual matrices between contrasts 1,000 times to generate a null distribution of NMI values for each contrast. Matrices between groups were randomly shuffled and partitioned into functional networks, and NMI was calculated.   
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.normalized_mutual_info_score.html#sklearn.metrics.normalized_mutual_info_score

## Connectivity Strength
Caluclate Euclidean distance for each ROI-ROI pair. Linear regression with distance as a predictor of connectivity strength between groups.

### Within network changes
All within network pairwise relationships were averaged per group. Two-tailed T-test to assess differences. Bonferroni corrections as needed.

### Between network changes
Average connectivity is calculated per network. Compare the between network interactions. 

## Participation Coefficient
Partition networks into the modules, calculate the PC per node within each group. Higher PC indicates more distributed between network connectivity, while a PC of 0 signifies a node’s links are completely within its home network (within network).

## Pickling data to save it

In [ ]:
# sfile = bz2.BZ2File('/Users/gracer/Google Drive/ABCD/tmp/smallerfile', 'w')
# pickle.dump(GRAPHS, sfile)
# z['allCC'] = allCC
# z['allPC'] = allPC
# pickle.dump(z, open('/Users/gracer/Google Drive/ABCD/tmp/current', 'wb'), protocol=4)